In [ ]:
import requests
import tabula
import pandas as pd

# Download of Module Handbooks

In [ ]:
pdf_url_bachelor_business_management_and_economics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-184-H-2008.pdf"
pdf_url_bachelor_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-277-H-2021.pdf"
pdf_url_bachelor_economathematics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-276-H-2015.pdf"
pdf_url_master_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-j10-H-2019.pdf"
pdf_url_master_management ="https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-i45-H-2018.pdf"
pdf_url_master_international_economic_policy = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-h92-H-2018.pdf"

response = requests.get(pdf_url_bachelor_business_management_and_economics)
with open("module_handbooks/bachelor_business_management_and_economics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_business_management_and_economics.pdf")

response = requests.get(pdf_url_bachelor_information_systems)
with open("module_handbooks/bachelor_information_systems.pdf", "wb") as f:
    f.write(response.content)

print("Downloaded bachelor_information_systems.pdf")
response = requests.get(pdf_url_bachelor_economathematics)
with open("module_handbooks/bachelor_economathematics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_economathematics.pdf")

response = requests.get(pdf_url_master_information_systems)
with open("module_handbooks/master_information_systems.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_information_systems.pdf")

response = requests.get(pdf_url_master_management)
with open("module_handbooks/master_management.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_management.pdf")

response = requests.get(pdf_url_master_international_economic_policy)
with open("module_handbooks/master_international_economic_policy.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_international_economic_policy.pdf")

# Ansatz über TABULA --> Verforfen 
Reading Tables from Module Book for master_information_systems

In [ ]:
import pandas as pd
import tabula
import re

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:


tables_master_is = tabula.read_pdf("module_handbooks/master_information_systems.pdf", pages='10-30', multiple_tables=True)
print(len(tables_master_is))
print(tables_master_is)

# PDF Miner Ansatz wie Felix -> über Regex

In [57]:
import io
import pdfminer
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage


def pdf_to_text(file_path, n=0):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    laparams = LAParams(all_texts=False, boxes_flow=[(0, 0, 0.97, 0.85), (0, 0.96, 0.8, 0.8)])
    converter = TextConverter(
        resource_manager,
        fake_file_handle,
        codec="utf-8",
        laparams=laparams,
    )
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(file_path, "rb") as fh:
        for i, page in enumerate(PDFPage.get_pages(fh, caching=True, check_extractable=True)):
            if i < n:
                continue
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    # close open handles
    converter.close()
    fake_file_handle.close()

    if text:
        return text
    else:
        return ""


In [58]:
text = pdf_to_text("module_handbooks/master_information_systems.pdf", n=10)
print(text)

TypeError: LAParam boxes_flow should be None, or a number between -1 and +1

In [ ]:
# remove footer and 
text_processed = re.sub(r"Master’s with 1 major Information Systems \(2019\).*?page \d+ / \d+", "", text)



# store the processd text in a new file
with open("module_handbooks/master_information_systems_processed.txt", "w") as f:
    f.write(text_processed)


In [ ]:
tables = re.split(r"Module title", text_processed)[1:]
print(len(tables))
print("---------------TABLE 0--------------------------")
print(tables[0])
print("---------------TABLE 1--------------------------")
print(tables[1])
print("---------------TABLE 2--------------------------")
print(tables[2])


# Create Finale Dataframe with these columns

In [ ]:
module_handbook_columns = [
    "module title",
    "abbreviation",
    "module coordinator",
    "module offered by",
    "ects",
    "method of grading",
    "only after successfull completion of module(s)",
    "duration",
    "module level",
    "other prerequisites",
    "contents",
    "intended learning outcomes",
    "courses",
    "method of assessment",
    "allocation of places",
    "additional information",
    "workload",
    "teaching cycle",
    "referred to in lpo I",
    "page",
]

module_handbook_df = pd.DataFrame(columns=module_handbook_columns)
print(module_handbook_df)
print(module_handbook_df.shape)
module_handbook_df.to_csv(
    "module_handbooks/master_information_systems.csv", index=False
)